<a href="https://colab.research.google.com/github/navneeshkaur/ML-and-AI-Course/blob/master/LSH_from_Scratch_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implement LSH from scratch

In this assignment, you will implement LSH from scratch and predict the labels of the test data. You will then verify the correctness of the your implementation using a "grader" function/cell (provided by us) which will match your implmentation.

The grader fucntion would help you validate the correctness of your code. 

Please submit the final Colab notebook in the classroom ONLY after you have verified your code using the grader function/cell.


**NOTE: DO NOT change the "grader" functions or code snippets written by us.Please add your code in the suggested locations.**

Ethics Code:
1. You are welcome to read up online resources to implement the code. 
2. You can also discuss with your classmates on the implmentation over Slack.
3. But, the code you wirte and submit should be yours ONLY. Your code will be compared against other stduents' code and online code snippets to check for plagiarism. If your code is found to be plagiarised, you will be awarded zero-marks for all assignments, which have a 10% wieghtage in the final marks for this course.

## Reading the data from csv file

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive


In [ ]:
# Loading data from csv file
import pandas as pd
data_path = '/gdrive/MyDrive/PGD Assignment Notebooks/lsh_assignment_data.csv'

df = pd.read_csv(data_path)
df

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...
...,...,...
2220,NaN,cars pull down us retail figures us retail sal...
2221,NaN,kilroy unveils immigration policy ex-chatshow ...
2222,NaN,rem announce new glasgow concert us band rem h...
2223,NaN,how political squabbles snowball it s become c...


In [ ]:
# Data Overiview
df['category'].value_counts()

sport            509
business         508
politics         415
tech             399
entertainment    384
Name: category, dtype: int64

### Creating Train and Test Datasets

Note that the labels for test data will not be present in the dataset and hence they are mentioned as NaN.

In [ ]:
# The last 10 rows in the csv file are query points, so loading them into test data.
# And loading the reamining points to train_data for which labels are given.

train_data = df.iloc[:-10,:]
test_data = df.iloc[-10:,:]

In [ ]:
# For train_data here the labels are in the column named "category".
train_data

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...
...,...,...
2210,politics,teens know little of politics teenagers ques...
2211,entertainment,lopez misses uk charity premiere jennifer lope...
2212,business,christmas shoppers flock to tills shops all ov...
2213,tech,progress on new internet domains by early 2005...


In [ ]:
test_data

,category,text
2215,NaN,junk e-mails on relentless rise spam traffic i...
2216,NaN,top stars join us tsunami tv show brad pitt r...
2217,NaN,rings of steel combat net attacks gambling is ...
2218,NaN,davies favours gloucester future wales hooker ...
2219,NaN,beijingers fume over parking fees choking traf...
2220,NaN,cars pull down us retail figures us retail sal...
2221,NaN,kilroy unveils immigration policy ex-chatshow ...
2222,NaN,rem announce new glasgow concert us band rem h...
2223,NaN,how political squabbles snowball it s become c...
2224,NaN,souness delight at euro progress boss graeme s...


## Custom Implementation

In [ ]:
# Please implement this fucntion and write your code wherever asked. Do NOT change the code snippets provided by us.

import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
from numpy import dot
from numpy.linalg import norm

def predictLabels (test_data):
  np.random.seed(0)
  """
  Given the test_data, return the labels for all the rows in the test data.

  1. Read in the train_data.
  2. Vectorize train_data using sklearns built in tfidf vectorizer.
  3. Ignore unigrams and make use of both bigrams & trigrams and also limit the max features to 4000 and minimum document frequency to 10.
  4. After the tfidf vectors are generated as mentioned above, next task is to generate random hyperplanes.
  5. Generate 5 random hyperplanes. And generate the hyperplanes using a random normal distribution with mean zero and variance 1. 
  6. We have set the numpy random seed to zero, please do not change it. And then you can make use of np.random.normal to generate those vectors.
  7. As mentioned in the course videos, compute the hash function and also the corresponding hash table for it.
  8. Once the hash table is generated now take in each of the query points from the test data.
  9. Vectorize those query points using the same tfidf vectorizer as mentioned above.
  10. Now use the hash function on this query point and fetch all the similar data points from the hashtable.
  11. Use cosine similarity to compute 11-Nearest Neighbours from the list of data points obtained in the above step.
  12. Take a majority vote among the 11-Nearest Neighbours and predict the class label for the query point in the test data.
  13. In case of a tie in the obtained labels from nearest neighbours, you can pick one alphabetically.
  14. Repeat steps 8 to 11 for all the points in the test data and then finally return a list with all the predicted labels.
  15. Note that there are a total of 10 data points in the test data so the final list you return should be of length 10.
  16. Also note that the cosine similarity function should be written from scratch, you should not directly make use of existing libraries.
  17. Please use the formula of cosine similarity as explained in the course videos, you can make use of numpy or scipy to calculate dot or norm or transpose.

  """

  
  ##############################################################
  ####   YOUR CODE BELOW  as per the above instructions #######
  ##############################################################

  #defining the required parameters
  max_features=4000 #max features for tf-idf
  ngram_min=2 #min n_gram allowed
  ngram_max=3 # max n_gram allowed
  min_df=10 #min_df value for tf-idf vectorizer
  k_neigh=11 #number of nearest neighbours to consider from the common bin
  num_hyper_planes=5 #number of random hyper planes to be generated


  #generating train tf-idf vectors
  # ***** COMMONS MISTAKE: Do NOT mix train and test data to generate feature vectors *******
  vectorizer = TfidfVectorizer(ngram_range=(ngram_min,ngram_max),max_features=max_features,min_df=min_df)
  train_vectors= vectorizer.fit_transform(train_data.text)


  #generating random hyper planes
  random_hyper_planes=[]
  for i in range(5):
    random_hyper_planes.append(np.random.normal(0,1,train_vectors.shape[1]))



  #initializing bins to store the points with the same hash value, 
  #number of bins is equal to the 2^num_hyper_planes, since each hyper plane has two sides i.e positive and negative
  bins=dict()
  for i in range(2**num_hyper_planes):
      bins[i]=[]




  #placing train data points inside bins, by looping through each datapoint and finding the direction of it with respect to each random plane.
  # tqdm: progress bar REFER: https://pypi.org/project/tqdm/
  for i in tqdm(range(train_vectors.shape[0])):
      bin_cal=0
      for j in range(len(random_hyper_planes)):
        if np.dot(random_hyper_planes[j],train_vectors[i][0].toarray()[0])>=0.0:
          bin_cal+=2**j # 
      bins[bin_cal].append((train_vectors[i],train_data.category.values[i]))
  


  #generating test tf-idf vectors
  test_vectors=vectorizer.transform(test_data.text)


  
  #for each test data point finding the bin it belongs to and  find k-nearest neighbours using cosine similarity
  preds=[]
  #looping through each test datapoint and finding the bins it belongs, by finding the direction of it with respect to each of the random planes
  for i in tqdm(range(test_vectors.shape[0])):
      bin_cal=0
      for j in range(len(random_hyper_planes)):
        if np.dot(random_hyper_planes[j],test_vectors[i][0].toarray()[0])>=0.0:
          bin_cal+=2**j

      #extracting all the train points that are in the same bin as the current test data point.
      neighbours=list(bins[bin_cal])

      #finding the cosine similarities of all train data points in the bin with the current test data point.
      consine_similarities=[]
      for j in range(len(neighbours)):
        cos_sim = dot(neighbours[j][0].toarray()[0], test_vectors[i][0].toarray()[0])/(norm(neighbours[j][0].toarray()[0])*norm(test_vectors[i][0].toarray()[0]))
        consine_similarities.append(cos_sim)
      
      #taking the nearest k_neigh in the same bin
      nearest_k=np.argsort(consine_similarities)[::-1][:k_neigh]
      preds.append([neighbours[k][1]  for k in nearest_k])



  #defining the class_labels present in the data and sorting them in alphabetical order to enable us to do give preference in alphabetical order
  class_labels=sorted(list(set(train_data.category.values)))


  #finding the number of votes each class label has got for each test data point
  test_pred_counts=[]
  #looping through test predictions
  for i in range(len(preds)):
    exc=[]
    #looping through class labels and getting the number of votes for it for a particular test data point
    for j in class_labels:
        exc.append(preds[i].count(j))
    test_pred_counts.append(exc)



  #finding the max number of votes recieved for any class-label for each test datapoint.
  test_pred_counts=np.array(test_pred_counts)
  max_vals=np.max(np.array(test_pred_counts),axis=1)
  
  
  #finding the max voted class-label for each test datapoint and giving preference according to the alphabetical order by looping through class_labels that are alphabetically sorted
  class_label_predictions=[]
  #looping through test prediction counts
  for i in range(len(test_pred_counts)):
    #looping through the votes of each class_label
    for j in range(len(class_labels)):
      #checking whether the class_label is the max voted one for the given test data point.
      if test_pred_counts[i][j]==max_vals[i]:
        max_voted_class_label=class_labels[j]
        break
    class_label_predictions.append(max_voted_class_label)

  #returning the final class label predictions
  return class_label_predictions


## Grader Cell

Please execute the following Grader cell to verify the correctness of your above implementation. This cell will print "Success" if your implmentation of the predictLabels() is correct, else, it will print "Failed". Make sure you get a "Success" before you submit the code in the classroom.

In [ ]:
###########################################
## GRADER CELL: Do NOT Change this.
# This cell will print "Success" if your implmentation of the computeTFIDF() is correct.
# Else, it will print "Failed"
###########################################
import numpy as np

# compute TF-IDF using the computeTFIDF() function
Y_custom = np.array(predictLabels(test_data))

# Reference grader array - DO NOT MODIFY IT
Y_grader = np.array(['tech', 'entertainment', 'tech', 'sport', 'business', 'business', 'politics', 'entertainment', 'politics', 'sport'])

#calculating accuracy by comparing Y_grader and Y_custom
accuracy=np.sum(Y_grader==Y_custom)*10
# compare Y_grader and Y_custom
if accuracy>=80:
  print("******** Success ********","Accuracy Achieved",accuracy,'%')
else:
  print("####### Failed #######")
  print("\Y_grader = \n\n", Y_grader)
  print("\n","*"*50)
  print("\Y_custom = \n\n", Y_custom)


100%|██████████| 10/10 [00:00<00:00, 25.67it/s]


******** Success ******** Accuracy Achieved 90 %
